In [1]:
from os.path import dirname, join
import pandas as pd
from bokeh.io import curdoc

import numpy as np
from bokeh.layouts import row, widgetbox
from bokeh.models import ColumnDataSource, CustomJS, BoxSelectTool, BoxZoomTool, LassoSelectTool
from bokeh.models.widgets import RangeSlider, Button, DataTable, TableColumn, NumberFormatter
from bokeh.io import output_notebook, show, push_notebook
from bokeh.sampledata.methylation import data
from bokeh.models.widgets import Slider, TextInput

from bokeh.layouts import gridplot
from bokeh.plotting import figure, show, output_file
from bokeh.models import Range1d

output_notebook()

Loading BokehJS ...

In [2]:
def filter_zero(originalArray):
    for i in range(originalArray.shape[0]):
         if originalArray[i]==0.0:
                originalArray[i]='nan'
    return originalArray

In [3]:
df = data.copy()

source_chromosome1 = ColumnDataSource(
    data=dict(
        # x-axis (position) for the chart
        x=df.position.values,
        # y-axis (pdf) for the chart
        y_methyPDF=filter_zero(df.methyPDF.values),
        y_X_PDF=filter_zero(df.X_PDF.values),
        y_H_PDF=filter_zero(df.H_PDF.values),
        y_Z_PDF=filter_zero(df.Z_PDF.values),
        y_U_PDF=filter_zero(df.U_PDF.values),
    )
)

In [4]:
TOOLS = "pan,wheel_zoom,box_zoom,reset,save,box_select,lasso_select"
plot = figure(title="TEST", tools=TOOLS, plot_width=800, plot_height=400)
#plot.background_fill_color = "beige"
plot.xaxis.axis_label = 'position'
plot.yaxis.axis_label = 'probability density function'
plot.y_range = Range1d(0, 1.1)

plot.circle('x', 'y_methyPDF', color="pink", legend="methyPDF", source=source_chromosome1)
plot.diamond('x', 'y_X_PDF', color="blue", legend="X_PDF", source=source_chromosome1)
plot.cross('x', 'y_H_PDF', color="red", legend="H_PDF", source=source_chromosome1)
plot.square('x', 'y_Z_PDF', color="yellow", legend="Z_PDF", source=source_chromosome1)
plot.triangle('x', 'y_U_PDF', color="green", legend="U_PDF", source=source_chromosome1)

plot.legend.location = "top_left"
plot.legend.click_policy="hide"
plot.legend.background_fill_color = "beige"
plot.legend.background_fill_alpha = 0.5

In [5]:
# Set up callbacks
def update_title(attrname, old, new):
    plot.title.text = text.value

def update_data_by_number():
    current = df[(df['totalNum'] >= totalNum.value[0]) & (df['totalNum'] <= totalNum.value[1])]
    
    x=current.position.values
    y_methyPDF=current.methyPDF.values 
    y_X_PDF=current.X_PDF.values
    y_H_PDF=current.H_PDF.values
    y_Z_PDF=current.Z_PDF.values
    y_U_PDF=current.U_PDF.values
    source_chromosome1.data = dict(x=x, y_methyPDF=y_methyPDF, y_X_PDF=y_X_PDF, y_H_PDF=y_H_PDF, y_Z_PDF=y_Z_PDF, y_U_PDF=y_U_PDF)
    
def update_data_by_position():
    current = df[(df['position'] >= positionRange.value[0]) & (df['position'] <= positionRange.value[1])]
      
    x=current.position.values
    y_methyPDF=current.methyPDF.values  
    y_X_PDF=current.X_PDF.values
    y_H_PDF=current.H_PDF.values
    y_Z_PDF=current.Z_PDF.values
    y_U_PDF=current.U_PDF.values
    source_chromosome1.data = dict(x=x, y_methyPDF=y_methyPDF, y_X_PDF=y_X_PDF, y_H_PDF=y_H_PDF, y_Z_PDF=y_Z_PDF, y_U_PDF=y_U_PDF)

In [6]:
# Set up widgets
text = TextInput(title="title", value='test1')
totalNum = RangeSlider(title="Total Number", start=0, end=9999, value=(8,500), step=5, format="0,0")
positionRange = RangeSlider(title="Position Range", start=10000000, end=22000000, value=(12068000,12078000), step=5, format="0,0")
text.on_change('value', update_title)
totalNum.on_change('value', lambda attr, old, new: update_data_by_number())
positionRange.on_change('value', lambda attr, old, new: update_data_by_position())

In [7]:
# Set up layouts and add to document
inputs = widgetbox(text, totalNum, positionRange)

In [8]:
def test(doc):
    
    update_data_by_number()
    update_data_by_position()
    doc.add_root(row(inputs, plot, width=800))
    doc.title = "Sliders"

In [9]:
show(test)